In [1]:
import os
import xml.etree.ElementTree as ET
from PIL import Image

In [3]:
import os
import xml.etree.ElementTree as ET
from PIL import Image

def get_image_dimensions(image_path):
    """
    Get image dimensions (width, height) using PIL.

    Parameters:
    - image_path: Path to the image file.

    Returns:
    - A tuple (width, height) of the image dimensions.
    """
    with Image.open(image_path) as img:
        return img.width, img.height

def convert_xml_to_yolo(xml_file_path, img_width, img_height):
    """
    Convert XML annotation to YOLO format.

    Parameters:
    - xml_file_path: Path to the XML file.
    - img_width: Width of the image.
    - img_height: Height of the image.

    Returns:
    - A list of strings, each representing an object in YOLO format.
    """
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    yolo_data = []
    for member in root.findall('object'):
        class_name = member.find('name').text
        class_id = class_names.index(class_name)  # Adjust based on your classes
        
        bndbox = member.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        x_center = ((xmin + xmax) / 2) / img_width
        y_center = ((ymin + ymax) / 2) / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

#         yolo_data.append(f"{class_id} {x_center} {y_center} {width} {height}")
        yolo_data.append(f"{class_id} {float(x_center):.3f} {float(y_center):.3f} {float(width):.3f} {float(height):.3f}")

    return yolo_data

# Define the path to your dataset
dataset_path = '/home/anil/Downloads/Helmet/archive/'
images_dir = os.path.join(dataset_path, 'images')
annotations_dir = os.path.join(dataset_path, 'annotations')
base_output_dir = '/home/anil/Downloads/Helmet/archive/'
annotations_output_dir = os.path.join(base_output_dir, 'annotations_txt')

# Define your class names based on your dataset
class_names = ['With Helmet', 'Without Helmet']  # Example class names
os.makedirs(annotations_output_dir, exist_ok=True)

for xml_file in os.listdir(annotations_dir):
    if xml_file.endswith('.xml'):
        img_file = xml_file.replace('.xml', '.png')  # Adjust if your image format differs
        img_path = os.path.join(images_dir, img_file)
        
        img_width, img_height = get_image_dimensions(img_path)
        
        yolo_annotations = convert_xml_to_yolo(os.path.join(annotations_dir, xml_file), img_width, img_height)
        
        # Save YOLO annotations to a .txt file in the annotations directory
        txt_file_path = os.path.join(annotations_output_dir, xml_file.replace('.xml', '.txt'))
        with open(txt_file_path, 'w') as file:
            for line in yolo_annotations:
                file.write(line + "\n")
